In [36]:
# import library
import random
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm
import lightgbm as lgb
import tqdm as tq
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

# set seed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) #

In [37]:
# data load
train_origin = pd.read_csv('/Users/idoyun/Desktop/데이터 분석 경진대회/진행중/2023 전력사용량 예측 AI 경진대회/datafile/train.csv')
test_origin = pd.read_csv('/Users/idoyun/Desktop/데이터 분석 경진대회/진행중/2023 전력사용량 예측 AI 경진대회/datafile/test.csv')
building_info = pd.read_csv('/Users/idoyun/Desktop/데이터 분석 경진대회/진행중/2023 전력사용량 예측 AI 경진대회/datafile/building_info.csv')

In [38]:
train_df = train_origin.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption'
})
train_df.drop('num_date_time', axis = 1, inplace=True)

In [39]:
test_df = test_origin.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption'
})
test_df.drop('num_date_time', axis = 1, inplace=True)

In [40]:
building_info = building_info.rename(columns={
    '건물번호': 'building_number',
    '건물유형': 'building_type',
    '연면적(m2)': 'total_area',
    '냉방면적(m2)': 'cooling_area',
    '태양광용량(kW)': 'solar_power_capacity',
    'ESS저장용량(kWh)': 'ess_capacity',
    'PCS용량(kW)': 'pcs_capacity'
})

In [41]:
translation_dict = {
    '건물기타': 'Other Buildings',
    '공공': 'Public',
    '대학교': 'University',
    '데이터센터': 'Data Center',
    '백화점및아울렛': 'Department Store and Outlet',
    '병원': 'Hospital',
    '상용': 'Commercial',
    '아파트': 'Apartment',
    '연구소': 'Research Institute',
    '지식산업센터': 'Knowledge Industry Center',
    '할인마트': 'Discount Mart',
    '호텔및리조트': 'Hotel and Resort'
}

building_info['building_type'] = building_info['building_type'].replace(translation_dict)

In [42]:
train_df = pd.merge(train_df, building_info, on='building_number', how='left')
test_df = pd.merge(test_df, building_info, on='building_number', how='left')

In [43]:
# solar_power_capacity, ess_capacity, pcs_capacity 대부분 결측자료 drop
train_df = train_df.drop(['solar_power_capacity', 'ess_capacity', 'pcs_capacity'], axis=1)
test_df = test_df.drop(['solar_power_capacity', 'ess_capacity', 'pcs_capacity'], axis=1)
train_df = train_df.drop(['solar_radiation', 'sunshine'], axis=1)

In [44]:
columns_to_fill = ['rainfall', 'windspeed', 'humidity']
train_df[columns_to_fill] = train_df[columns_to_fill].fillna(0)
test_df[columns_to_fill] = test_df[columns_to_fill].fillna(0)

In [45]:
# 시계열 변수 생성
date = pd.to_datetime(train_df.date_time)
train_df['hour'] = date.dt.hour
train_df['day'] = date.dt.day
train_df['month'] = date.dt.month
train_df['week'] = date.dt.weekofyear
train_df['weekday'] = date.dt.weekday

/var/folders/d1/ckn606sd76d8426_12q7ccyh0000gn/T/ipykernel_96109/1854651519.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  train_df['week'] = date.dt.weekofyear


In [46]:
date = pd.to_datetime(test_df.date_time)
test_df['hour'] = date.dt.hour
test_df['day'] = date.dt.day
test_df['month'] = date.dt.month
test_df['week'] = date.dt.weekofyear
test_df['weekday'] = date.dt.weekday

/var/folders/d1/ckn606sd76d8426_12q7ccyh0000gn/T/ipykernel_96109/3234783852.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  test_df['week'] = date.dt.weekofyear


In [47]:
# building_type 변수를 수치형으로 변환하여 새로운 열 생성
building_type_mapping = {
    'Apartment': 1,
    'Commercial': 2,
    'Data Center': 3,
    'Department Store and Outlet': 4,
    'Discount Mart': 5,
    'Hospital': 6,
    'Hotel and Resort': 7,
    'Knowledge Industry Center': 8,
    'Other Buildings': 9,
    'Public': 10,
    'Research Institute': 11,
    'University': 12
}
train_df['building_type_numeric'] = train_df['building_type'].map(building_type_mapping)
test_df['building_type_numeric'] = test_df['building_type'].map(building_type_mapping)

In [48]:
### 공휴일 변수 추가
train_df['holiday'] = train_df['weekday'].apply(lambda x: 1 if x >= 6 else 0)
test_df['holiday'] = test_df['weekday'].apply(lambda x: 1 if x >= 6 else 0)

In [49]:
## https://dacon.io/competitions/official/235680/codeshare/2366?page=1&dtype=recent
# 순환적 성격의 시간 변수 생성
train_df['sin_time'] = np.sin(2*np.pi*train_df.hour/24)
train_df['cos_time'] = np.cos(2*np.pi*train_df.hour/24)

test_df['sin_time'] = np.sin(2*np.pi*test_df.hour/24)
test_df['cos_time'] = np.cos(2*np.pi*test_df.hour/24)

In [50]:
## https://dacon.io/competitions/official/235736/codeshare/2743?page=1&dtype=recent
# 상대습도 변수 생성
train_df['THI'] = 9/5*train_df['temperature'] - 0.55*(1-train_df['humidity']/100)*(9/5*train_df['humidity']-26)+32
test_df['THI'] = 9/5*test_df['temperature'] - 0.55*(1-test_df['humidity']/100)*(9/5*test_df['humidity']-26)+32

In [51]:
# Cooling Degree Hour 변수 생성
def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

cdhs = np.array([])
for num in range(1,101,1):
    temp = train_df[train_df['building_number'] == num]
    cdh = CDH(temp['temperature'].values)
    cdhs = np.concatenate([cdhs, cdh])
train_df['CDH'] = cdhs


cdhs = np.array([])
for num in range(1,101,1):
    temp = test_df[test_df['building_number'] == num]
    cdh = CDH(temp['temperature'].values)
    cdhs = np.concatenate([cdhs, cdh])
test_df['CDH'] = cdhs

In [52]:
train_df.head()

,building_number,date_time,temperature,rainfall,windspeed,humidity,power_consumption,building_type,total_area,cooling_area,...,day,month,week,weekday,building_type_numeric,holiday,sin_time,cos_time,THI,CDH
0,1,20220601 00,18.6,0.0,0.9,42.0,1085.28,Other Buildings,110634.0,39570.0,...,1,6,22,2,9,0,0.000000,1.000000,49.6576,-7.4
1,1,20220601 01,18.0,0.0,1.1,45.0,1047.36,Other Buildings,110634.0,39570.0,...,1,6,22,2,9,0,0.258819,0.965926,47.7625,-15.4
2,1,20220601 02,17.7,0.0,1.5,45.0,974.88,Other Buildings,110634.0,39570.0,...,1,6,22,2,9,0,0.500000,0.866025,47.2225,-23.7
3,1,20220601 03,16.7,0.0,1.4,48.0,953.76,Other Buildings,110634.0,39570.0,...,1,6,22,2,9,0,0.707107,0.707107,44.7856,-33.0
4,1,20220601 04,18.4,0.0,2.8,43.0,986.40,Other Buildings,110634.0,39570.0,...,1,6,22,2,9,0,0.866025,0.500000,49.0061,-40.6


In [53]:
test_df.head()

,building_number,date_time,temperature,rainfall,windspeed,humidity,building_type,total_area,cooling_area,hour,day,month,week,weekday,building_type_numeric,holiday,sin_time,cos_time,THI,CDH
0,1,20220825 00,23.5,0.0,2.2,72,Other Buildings,110634.0,39570.0,0,25,8,34,3,9,0,0.000000,1.000000,58.3456,-2.5
1,1,20220825 01,23.0,0.0,0.9,72,Other Buildings,110634.0,39570.0,1,25,8,34,3,9,0,0.258819,0.965926,57.4456,-5.5
2,1,20220825 02,22.7,0.0,1.5,75,Other Buildings,110634.0,39570.0,2,25,8,34,3,9,0,0.500000,0.866025,57.8725,-8.8
3,1,20220825 03,22.1,0.0,1.3,78,Other Buildings,110634.0,39570.0,3,25,8,34,3,9,0,0.707107,0.707107,57.9376,-12.7
4,1,20220825 04,21.8,0.0,1.0,77,Other Buildings,110634.0,39570.0,4,25,8,34,3,9,0,0.866025,0.500000,56.9961,-16.9


In [54]:
# train test split
def train_test_split(df, th):
    train = df[df['date_time'].str[:8].astype(int) < th].reset_index(drop=True)
    test = df[df['date_time'].str[:8].astype(int) >= th].reset_index(drop=True)
    return train, test

In [55]:
# 2022년 8월 20일 기준으로 train data test data split
date_th = 20220820

In [56]:
train_df, valid_df = train_test_split(train_df, date_th)

In [57]:
train_x = train_df.drop(columns = ['power_consumption','date_time','total_area','cooling_area','building_type',
                                  'building_type_numeric'])
train_y = train_df['power_consumption']

valid_x = valid_df.drop(columns = ['power_consumption','date_time','total_area','cooling_area','building_type',
                                  'building_type_numeric'])
valid_y = valid_df['power_consumption']

In [58]:
# function define
def SMAPE(y, pred):
    smape = abs((y - pred))/((abs(y) + abs(pred)) / 2) * 100
    smape = np.mean(smape)
    return smape

def mae(y, pred):
    return np.mean(abs(y-pred))

In [59]:
def validate(valid_x, valid_y, model):
    pred = model.predict(valid_x)
    smape_score, mae_score = SMAPE(valid_y, pred), mae(valid_y, pred)
    return smape_score, mae_score

## XGBoost 

In [25]:
# modeling
model_xgb = XGBRegressor()
model_xgb.fit(train_x, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [26]:
smape_score, mae_score = validate(valid_x, valid_y, model_xgb)
print(f'SMAPE: {smape_score}\nMAE: {mae_score}')

SMAPE: 12.141220918997115
MAE: 226.54799858433915


## XGBoost ( modeling each building)

In [60]:
def validate_multi(valid_x, valid_y, models):
    preds = []
    for i in range(1, 101):
        _x = valid_x[valid_x['building_number'] == i]
        _x = _x.drop(columns=['building_number'])
        pred = models[i].predict(_x).tolist()
        preds.extend(pred)
    preds = np.array(preds)
    smape_score, mae_score = SMAPE(valid_y, preds), mae(valid_y, preds)
    return smape_score, mae_score

In [61]:
def train_multiple_models(train_x, train_y, n_estimators=100):
    models = {}
    for i in tqdm(range(1, 101)):
        _x = train_x[train_x['building_number'] == i]
        _x = _x.drop(columns=['building_number'])
        _y = train_y[_x.index]
        
        param_grid = {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.2]}

        # Create the XGBoost regressor
        model_xgb = XGBRegressor(verbose=-1)

        # Create GridSearchCV
        grid_search = GridSearchCV(model_xgb, param_grid, scoring='neg_mean_absolute_error', cv=3)
        grid_search.fit(_x, _y)

        # Get the best estimator from grid search
        best_model = grid_search.best_estimator_

        models[i] = best_model

    return models

In [67]:
models = train_multiple_models(train_x, train_y)

  0%|                                                   | 0/100 [00:00<?, ?it/s]

[17:13:49] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "verbose" } are not used.

[17:13:49] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "verbose" } are not used.

[17:13:49] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "verbose" } are not used.

[17:13:50] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "verbose" } are not used.

[17:13:50] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "verbose" } are not used.

[17:13:50] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x8

  1%|▍                                          | 1/100 [00:08<14:45,  8.95s/it]

[17:13:58] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "verbose" } are not used.

[17:13:58] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "verbose" } are not used.

[17:13:58] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "verbose" } are not used.

[17:13:58] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "verbose" } are not used.

[17:13:58] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "verbose" } are not used.

[17:13:59] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x8

  2%|▊                                          | 2/100 [00:17<13:48,  8.45s/it]

...

 98%|█████████████████████████████████████████▏| 98/100 [14:03<00:27, 13.84s/it]

[17:27:53] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "verbose" } are not used.

[17:27:53] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "verbose" } are not used.

[17:27:53] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "verbose" } are not used.

[17:27:53] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "verbose" } are not used.

[17:27:53] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "verbose" } are not used.

[17:27:54] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x8

 99%|█████████████████████████████████████████▌| 99/100 [14:14<00:12, 12.97s/it]

[17:28:03] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "verbose" } are not used.

[17:28:04] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "verbose" } are not used.

[17:28:04] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "verbose" } are not used.

[17:28:04] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "verbose" } are not used.

[17:28:04] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "verbose" } are not used.

[17:28:05] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x8

100%|█████████████████████████████████████████| 100/100 [14:25<00:00,  8.65s/it]


In [68]:
smape_score, mae_score = validate_multi(valid_x, valid_y, models)
print(f'SMAPE: {smape_score}\nMAE: {mae_score}')

SMAPE: 6.048766457226285
MAE: 117.0405819863885


In [69]:
test_x = test_df.drop(columns = ['date_time','total_area','cooling_area','building_type',
                                  'building_type_numeric'])

In [70]:
test_x.head()

,building_number,temperature,rainfall,windspeed,humidity,hour,day,month,week,weekday,holiday,sin_time,cos_time,THI,CDH
0,1,23.5,0.0,2.2,72,0,25,8,34,3,0,0.000000,1.000000,58.3456,-2.5
1,1,23.0,0.0,0.9,72,1,25,8,34,3,0,0.258819,0.965926,57.4456,-5.5
2,1,22.7,0.0,1.5,75,2,25,8,34,3,0,0.500000,0.866025,57.8725,-8.8
3,1,22.1,0.0,1.3,78,3,25,8,34,3,0,0.707107,0.707107,57.9376,-12.7
4,1,21.8,0.0,1.0,77,4,25,8,34,3,0,0.866025,0.500000,56.9961,-16.9


In [71]:
preds = []
for i in tqdm(range(1, 101)):
    _x = test_x[test_x['building_number'] == i]
    _x = _x.drop(columns=['building_number'])
    pred = models[i].predict(_x).tolist()
    preds.extend(pred)

100%|█████████████████████████████████████████| 100/100 [00:01<00:00, 96.27it/s]


In [72]:
submission = pd.read_csv("/Users/idoyun/Desktop/데이터 분석 경진대회/진행중/2023 전력사용량 예측 AI 경진대회/datafile/sample_submission.csv")

In [73]:
submission['answer'] = preds

In [75]:
submission

,num_date_time,answer
0,1_20220825 00,1845.691284
1,1_20220825 01,1948.210083
2,1_20220825 02,1695.956665
3,1_20220825 03,1808.871460
4,1_20220825 04,1815.224487
...,...,...
16795,100_20220831 19,955.826050
16796,100_20220831 20,764.440735
16797,100_20220831 21,704.879944
16798,100_20220831 22,597.901672


In [76]:
submission.to_csv('/Users/idoyun/Desktop/데이터 분석 경진대회/진행중/2023 전력사용량 예측 AI 경진대회/datafile/baseline_submission_xgboost_hpyerparameter.csv', index=False)